In [9]:
conda install -c conda-forge implicit

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Pedro\anaconda3

  added / updated specs:
    - implicit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2020.07  |           py38_0           6 KB
    anaconda-custom            |           py38_1          36 KB
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    gmpy2-2.1.0b1              |   py38hb63f22f_1         192 KB  conda-forge
    implicit-0.4.4             |   py38hddbc854_3         499 KB  conda-forge
    libllvm9-9.0.1             |       hab3b255_3          48 KB  conda-forge
    mpc-1.1.0                  |    h7edee0f_1009         322 KB  conda-forge
    mpfr-4.0.2                 |       h62dcd97_1         1.9 MB  conda-forge
    mpir-3.0.0                 |    he025d50_1002         3.0 MB  conda-forge
    pip-21.1               


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2020.11=py38_0
  - defaults/win-64::bcrypt==3.2.0=py38he774522_0
  - defaults/win-64::ipykernel==5.3.4=py38h5ca1d4c_0
  - defaults/win-64::ipython==7.19.0=py38hd4e2768_0
  - defaults/noarch::ipywidgets==7.5.1=py_1
  - defaults/win-64::jupyter==1.0.0=py38_7
  - defaults/noarch::jupyterlab==2.2.6=py_0
  - defaults/noarch::jupyterlab_server==1.2.0=py_0
  - defaults/noarch::jupyter_console==6.2.0=py_0
  - defaults/win-64::notebook==6.1.4=py38_0
  - defaults/noarch::paramiko==2.7.2=py_0
  - defaults/noarch::prompt_toolkit==3.0.8=0
  - defaults/noarch::qtconsole==4.7.7=py_0
  - defaults/win-64::spyder==4.1.5=py38_0
  - defaults/win-64::spyder-kernels==1.9.4=py38_0
  - defaults/win-64::widgetsnbextension==3.5.1=py38_0
  - defaults/win-64::_ipyw_jlab_nb_ext_conf==0.1.0=py38_0


==> WARNING: A newer version of conda exists. <==
  current

  mpir               conda-forge/win-64::mpir-3.0.0-he025d50_1002
  pip                conda-forge/noarch::pip-21.1-pyhd8ed1ab_0
  prompt-toolkit     conda-forge/noarch::prompt-toolkit-3.0.18-pyha770c72_0
  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38
  snappy             conda-forge/win-64::snappy-1.1.8-ha925a31_3
  tbb                conda-forge/win-64::tbb-2021.2.0-h2d74725_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2020.10.14~ --> conda-forge::ca-certificates-2020.12.5-h5b45459_0
  certifi            pkgs/main/noarch::certifi-2020.6.20-p~ --> conda-forge/win-64::certifi-2020.12.5-py38haa244fe_1
  conda               pkgs/main::conda-4.9.2-py38haa95532_0 --> conda-forge::conda-4.10.1-py38haa244fe_0
  openssl              pkgs/main::openssl-1.1.1h-he774522_0 --> conda-forge::openssl-1.1.1k-h8ffe710_0
  prompt_toolkit          pkgs/main::prompt_toolkit-3.0.8-0 --> conda-forge::prompt_toolkit-3.0.18-hd8ed1ab_0

The follow

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import ranking_metrics_at_k
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

In [11]:
# Read csv files
df = pd.read_csv('retail.csv')

In [12]:
df.shape

(541909, 8)

In [13]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [14]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [15]:
# Get df_events and correctly encode the variables
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],unit='ms')
df = df.astype({'CustomerID': object, 'StockCode': object})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [16]:
df.rename(columns ={'InvoiceNo':'TransactionId', 'StockCode':'ItemId', 'InvoiceDate':'TransactionDate'},inplace=True)

In [17]:
# Describe dataset
df.describe(include='all', datetime_is_numeric=True)

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909.000000,541909,541909.000000,406829.0,541909
unique,25900,4070,4223,NaN,NaN,NaN,4372.0,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,17841.0,United Kingdom
freq,1114,2313,2369,NaN,NaN,NaN,7983.0,495478
mean,NaN,NaN,NaN,9.552250,2011-07-04 13:34:57.156151552,4.611114,NaN,NaN
min,NaN,NaN,NaN,-80995.000000,2010-12-01 08:26:00,-11062.060000,NaN,NaN
25%,NaN,NaN,NaN,1.000000,2011-03-28 11:34:00,1.250000,NaN,NaN
50%,NaN,NaN,NaN,3.000000,2011-07-19 17:17:00,2.080000,NaN,NaN
75%,NaN,NaN,NaN,10.000000,2011-10-19 11:27:00,4.130000,NaN,NaN
max,NaN,NaN,NaN,80995.000000,2011-12-09 12:50:00,38970.000000,NaN,NaN


In [18]:
# Percentage of missing values 
(df.isna().sum() / df.shape[0])* 100

TransactionId       0.000000
ItemId              0.000000
Description         0.268311
Quantity            0.000000
TransactionDate     0.000000
UnitPrice           0.000000
CustomerID         24.926694
Country             0.000000
dtype: float64

In [21]:
df[df.CustomerID.isna()]

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [33]:
df

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [103]:
top10prod=df["Description"].value_counts().to_frame()
top10prod = top10prod.rename(columns={"Description":"Total Amount of Sales"})
top10prod

,Total Amount of Sales
WHITE HANGING HEART T-LIGHT HOLDER,2369
REGENCY CAKESTAND 3 TIER,2200
JUMBO BAG RED RETROSPOT,2159
PARTY BUNTING,1727
LUNCH BAG RED RETROSPOT,1638
...,...
CHRISTMAS TABLE CANDLE SILVER SPIKE,1
WHITE FRANGIPANI HAIR CLIP,1
OOH LA LA DOGS COLLAR,1
water damaged,1


In [102]:
len(top10prod)

15

In [49]:
fig = px.bar(top10prod, x=top10prod.index, y='Total Amount of Sales', color='Total Amount of Sales')
fig.update_traces(showlegend = False)
fig.update_layout(barmode='stack', paper_bgcolor='rgba(255,255,255)', plot_bgcolor='rgba(0,0,0,0)', xaxis_title="Product Name",
    yaxis_title="Total Amount of Sales")
fig.update_traces(
        hovertemplate='Product: %{x} <br>Total Sales: %{y} <extra></extra>')
fig.update_yaxes(gridcolor='black', showgrid=True, gridwidth=0.5)
fig.show()

In [106]:
prices = df["UnitPrice"].value_counts().to_frame()
prices = prices.rename(columns={"UnitPrice":"Amount of Sales for Price"})
prices["Prices"] = prices.index
prices

,Amount of Sales for Price,Prices
1.25,50496,1.25
1.65,38181,1.65
0.85,28497,0.85
2.95,27768,2.95
0.42,24533,0.42
...,...,...
4575.64,1,4575.64
125.96,1,125.96
147.13,1,147.13
3060.60,1,3060.60


In [112]:
num_cust=df["CustomerID"].value_counts()
num_cust

17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
13391.0       1
16579.0       1
13366.0       1
17763.0       1
17846.0       1
Name: CustomerID, Length: 4372, dtype: int64

In [81]:
max(prices["Prices"])

38970.0

In [77]:
import plotly.figure_factory as ff

fig = px.histogram(prices, x=prices.index, y="Amount of Sales for Price", color="Amount of Sales for Price", marginal="rug")
fig.show()

In [90]:
countries = df["Country"]
countries = countries.value_counts().to_frame()
countries = countries.rename(columns={"Country":"Amount of Sales for Country"})
countries

,Amount of Sales for Country
United Kingdom,495478
Germany,9495
France,8557
EIRE,8196
Spain,2533
Netherlands,2371
Belgium,2069
Switzerland,2002
Portugal,1519
Australia,1259


In [91]:
fig = px.bar(countries, x=countries.index, y='Amount of Sales for Country', color='Amount of Sales for Country')
fig.update_traces(showlegend = False)
fig.update_layout(barmode='stack', paper_bgcolor='rgba(255,255,255)', plot_bgcolor='rgba(0,0,0,0)', xaxis_title="Country",
    yaxis_title="Amount of Sales for Country")
fig.update_traces(
        hovertemplate='Country: %{x} <br> Sales for Country: %{y} <extra></extra>')
fig.update_yaxes(gridcolor='black', showgrid=True, gridwidth=0.5)
fig.show()

In [98]:
countries10 = countries.head(10)
fig = px.pie(countries10, values='Amount of Sales for Country', names=countries10.index, title='Total Sales for Top 10 Countries')
fig.update_traces(
        hovertemplate='')
fig.show()

In [119]:
df

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [120]:
df["hours"]=df["TransactionDate"].dt.hour

In [121]:
df

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country,hours
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,12


In [124]:
df['Day of Week'] = df['TransactionDate'].dt.day_name()

In [125]:
df

,TransactionId,ItemId,Description,Quantity,TransactionDate,UnitPrice,CustomerID,Country,hours,Day of Week
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,8,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,8,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,8,Wednesday
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,12,Friday
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12,Friday
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,12,Friday
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,12,Friday


In [128]:
grouped_df = df.groupby(["hours", "Day of Week"])["TransactionId"].aggregate("count").reset_index()

In [129]:
grouped_df

,hours,Day of Week,TransactionId
0,6,Thursday,41
1,7,Friday,134
2,7,Monday,61
3,7,Thursday,64
4,7,Tuesday,65
...,...,...,...
70,19,Thursday,3601
71,19,Wednesday,15
72,20,Friday,8
73,20,Thursday,845


In [143]:
fig = go.Figure(data=go.Heatmap(
          x = grouped_df["hours"],
          y = grouped_df["Day of Week"],
          z = grouped_df["TransactionId"],
          type = 'heatmap',
          colorscale = 'Viridis'))
fig.update_traces(
        hovertemplate='Hour: %{x} <br>Day of Week: %{y} <br>Sales: %{z} <extra></extra>')
fig.update_yaxes(gridcolor='black', showgrid=False, gridwidth=0.5)
fig.update_xaxes(gridcolor='black', showgrid=False, gridwidth=0.5)
fig.show()